In [ ]:
# install pysyft for federated learning
#!pip install syft[udacity]  

In [ ]:
## this is the module required for federated learning
#import syft as sy 

In [1]:
%tensorflow_version 1.x   #ensure that TF 1.x is used when TF 2.x becomes the default 

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
# import helper
import matplotlib.pyplot as plt
import numpy as np
import copy
import random

In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [6]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
                              ])
# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

In [ ]:
images.__len__()

In [ ]:
plt.imshow(images[1].numpy().squeeze(), cmap='Greys_r');

In [ ]:
 plt.imshow((image+torch.Tensor(np.random.normal(0.0, 0.25, (1,28,28)))).numpy().squeeze(), cmap='Greys_r')

In [ ]:
plt.imshow((image+torch.Tensor(np.random.normal(0.0, 0.5, (1,28,28)))).numpy().squeeze(), cmap='Greys_r')

# Structure of the MNIST dataset as retrieved by torchvision.datasets.MNIST
1. length of trainset: 60000 
2. each element of trainset is a tuple (image, label) where image is a 1x28x28 tensor and label is an integer in range 0-9 inclusive


# Splitting of Data
1. Same distribution with same data size: splitting data set into five parts with the same size randomly, ensuring number of images with the same label is mutually equal among the five parts

In [7]:
#create lists for sorting of data
trainset_0 = []
trainset_1 = []
trainset_2 = []
trainset_3 = []
trainset_4 = []
trainset_5 = []
trainset_6 = []
trainset_7 = []
trainset_8 = []
trainset_9 = []

In [8]:
#creates a dictionary called dictionaryData to sort the different labels
dictionaryData = {}
for k in range(10):
  dictionaryData['trainset_'+str(k)] = []

In [9]:
dictionaryData.keys()

dict_keys(['trainset_0', 'trainset_1', 'trainset_2', 'trainset_3', 'trainset_4', 'trainset_5', 'trainset_6', 'trainset_7', 'trainset_8', 'trainset_9'])

In [10]:
dictionaryData

{'trainset_0': [],
 'trainset_1': [],
 'trainset_2': [],
 'trainset_3': [],
 'trainset_4': [],
 'trainset_5': [],
 'trainset_6': [],
 'trainset_7': [],
 'trainset_8': [],
 'trainset_9': []}

In [13]:
trainset[2][1]

4

In [14]:
for i in range(60000):
  dictionaryData['trainset_'+str(trainset[i][1])].append(trainset[i])

In [ ]:
6000-1

In [ ]:
dictionaryData['trainset_0'+.append(trainset[i][0:5999])
dictionaryData['trainset_0'+.append(trainset[i][5999:5999+6000+1])
dictionaryData['trainset_0'+.append(trainset[i][5999:5999+6000+1])


In [ ]:
for i in range(10):
  print ("number of " + str(i) + ": " + str(len(dictionaryData['trainset_' + str(i)])))

#based on results, we will be using 5420 of each of the labels to setup our testbed.

In [ ]:
#creates a dictionary1 for splitting of data in case 1

dictionary1 = {}
for k in range(5):
  startIndex = 1084*k
  stopIndex = 1084*(k+1)
  dictionary1['trainset_1_'+str(k+1)] = []
  for i in range(10):
    dictionary1['trainset_1_'+str(k+1)].extend(dictionaryData['trainset_'+str(i)][startIndex:stopIndex])

In [ ]:
for i in range(5):
  print(len(dictionary1['trainset_1_'+str(i+1)]))

2. Different distribution with same data size. Split data set into five parts with same size, but labels are different. Let data provider 1 have 40% of label '0', 40% of label '1' and the data points with the remaining eight labels shared the remaining 20% of data provider 1's data. The rest shall be done in a same manner.

In [ ]:
#creates a dictionary2 for splitting of data in case 2

dictionary2 = {}
for k in range(5):
  dictionary2['trainset_2_'+str(k+1)] = []

#create data for trainset_2_1 which has 40% of 0 and 1
dictionary2['trainset_2_1'].extend(dictionaryData['trainset_0'][0:int(10840*0.4)]) # 0: indexes 0:4336  
dictionary2['trainset_2_1'].extend(dictionaryData['trainset_1'][0:int(10840*0.4)]) # 1: indexes 0:4336
for a in range(2,10):
  dictionary2['trainset_2_1'].extend(dictionaryData['trainset_'+str(a)][0:271]) # 2-9: indexes 0:271

#create data for trainset_2_2 which has 40% of 2 and 3
dictionary2['trainset_2_2'].extend(dictionaryData['trainset_0'][int(10840*0.4):int(10840*0.4)+271]) # 0: indexes 2168:2981
dictionary2['trainset_2_2'].extend(dictionaryData['trainset_1'][int(10840*0.4):int(10840*0.4)+271]) # 1: indexes 2168:2981
dictionary2['trainset_2_2'].extend(dictionaryData['trainset_2'][271:int(10840*0.4)+271]) #2: indexes 271:4607
dictionary2['trainset_2_2'].extend(dictionaryData['trainset_3'][271:int(10840*0.4)+271]) #3: indexes 271:4607
for b in range(4,10):
  dictionary2['trainset_2_2'].extend(dictionaryData['trainset_'+str(b)][271:(271*2)]) # 4-9: indexes 271:542

#create data for trainset_2_3 which has 40% of 4 and 5
for i in range(4):
  dictionary2['trainset_2_3'].extend(dictionaryData['trainset_'+str(i)][int(10840*0.4)+271:int(10840*0.4)+271*2]) #0-3: indexes 2981:3794
dictionary2['trainset_2_3'].extend(dictionaryData['trainset_4'][(271*2):int(10840*0.4)+271*2]) #4: indexes 1626:3794
dictionary2['trainset_2_3'].extend(dictionaryData['trainset_5'][(271*2):int(10840*0.4)+271*2]) #5: indexes 1626:3794
for c in range(6,10):
  dictionary2['trainset_2_3'].extend(dictionaryData['trainset_'+str(c)][271*2:(271*3)]) # 6-9: indexes 1626:2439

#create data for trainset_2_4 which has 40% of 6 and 7
for i in range(6):
  dictionary2['trainset_2_4'].extend(dictionaryData['trainset_'+str(i)][int(10840*0.4)+271*2:int(10840*0.4)+271*3]) #0-5: indexes 3794:4607
dictionary2['trainset_2_4'].extend(dictionaryData['trainset_6'][271*3:int(10840*0.4)+271*3]) #6: indexes 2439:4607
dictionary2['trainset_2_4'].extend(dictionaryData['trainset_7'][271*3:int(10840*0.4)+271*3]) #7: indexes 2439:4607
for d in range(8,10):
  dictionary2['trainset_2_4'].extend(dictionaryData['trainset_'+str(d)][271*3:(271*4)]) # 8-9: indexes 2439:3252

#create data for trainset_2_5 which has 40% of 8 and 9
for i in range(8):
  dictionary2['trainset_2_5'].extend(dictionaryData['trainset_'+str(i)][int(10840*0.4)+271*3:int(10840*0.4)+271*4])
for e in range(8,10):
  dictionary2['trainset_2_5'].extend(dictionaryData['trainset_'+str(e)][271*4:int(10840*0.4)+271*4])

for i in range(5):
  print("trainset_2_", str(i+1), len(dictionary2['trainset_2_'+str(i+1)]))

3. Same distribution with different data size. Split data set into five parts randomly, ratio of data size is 2:3:4:5:6. Ensure each part have the same number of images with different labels.

In [ ]:
#creates a dictionary3 for splitting of data in case 3

dictionary3 = {}
for k in range(5):
  dictionary3['trainset_3_'+str(k+1)] = []
for i in range(10):
  dictionary3['trainset_3_1'].extend(dictionaryData['trainset_'+str(i)][0:271*(2)])
  dictionary3['trainset_3_2'].extend(dictionaryData['trainset_'+str(i)][271*(2):271*(5)])
  dictionary3['trainset_3_3'].extend(dictionaryData['trainset_'+str(i)][271*(5):271*(9)])
  dictionary3['trainset_3_4'].extend(dictionaryData['trainset_'+str(i)][271*(9):271*(14)])
  dictionary3['trainset_3_5'].extend(dictionaryData['trainset_'+str(i)][271*(14):271*(20)])

for i in range(5):
  print(len(dictionary3['trainset_3_'+str(i+1)]))

4. Split data set into five parts with the same size randomly. Ensure number of images with the same label is mutually equal among the five parts. change the labels of different data providers randomly by 0%, 5%, 10%, 15% and 20%.

In [ ]:
dictionary4 = {}

In [ ]:
# deepcopy the data in dictionary1
for i in range(len(dictionary1)):
  dictionary4["trainset_4_"+str(i+1)] = copy.deepcopy(dictionary1["trainset_1_"+str(i+1)]) 

In [ ]:
percentageMislabel = {"trainset_4_1": 0, "trainset_4_2": 0.05, "trainset_4_3": 0.10, "trainset_4_4": 0.15, "trainset_4_5": 0.20}

In [ ]:
for i in range(len(dictionary4)):
  trainset_4_i = "trainset_4_" + str(i+1)
  numMislabel = int(percentageMislabel[trainset_4_i] * len(dictionary4[trainset_4_i]))
  print("For trainset_" + str(i) + ", number of mislabelled data is: ", numMislabel)
  indexesMislabel = random.sample(range(len(dictionary4[trainset_4_i])), numMislabel) # generate a random sample of indices without replacement)
  print(len(indexesMislabel))

  for k in indexesMislabel: 
    trueLabel = dictionary4[trainset_4_i][i][1]
    possibleMislabel = [x for x in range(10) if x != trueLabel] # list of all the possible mislabels to choose from
    dictionary4[trainset_4_i][k] = (dictionary4[trainset_4_i][k][0], random.choice(possibleMislabel)) # change the label for the k-th entry 
  print("Done", i)
    

In [ ]:
index = 6
plt.imshow(dictionary4["trainset_4_4"][index][0].numpy().squeeze(), cmap='Greys_r')
print("The label is :", dictionary4["trainset_4_4"][index][1])

5. Split data set into five parts with the same size. Generate 0-4x Gaussian noise where 0x is feature without noise

In [ ]:
dictionary5 = {}

In [ ]:
# deepcopy the data in dictionary1
for i in range(len(dictionary1)):
  dictionary5["trainset_5_"+str(i+1)] = copy.deepcopy(dictionary1["trainset_1_"+str(i+1)])

In [ ]:
percentageNoise = {"trainset_5_2": 0.25, "trainset_5_3": 0.5, "trainset_5_4": 0.75, "trainset_5_5": 1.0}
#trainset_5_1 is not varied with noise

In [ ]:
for i in range(1, len(dictionary5)):
  trainset_5_i = "trainset_5_" + str(i+1)
  for index in range(len(dictionary5[trainset_5_i])):
     dictionary5[trainset_5_i][index] = (dictionary5[trainset_5_i][index][0] + \
                                         torch.Tensor(np.random.normal(0.0, percentageNoise[trainset_5_i], (1,28,28))), \
                                         dictionary5[trainset_5_i][index][1])


In [ ]:
plt.imshow(dictionary5["trainset_5_2"][0][0].numpy().squeeze(), cmap='Greys_r')

In [ ]:
# root_dir = "./data/OurMNIST"
base_dir = "./data/OurMNIST/"

In [ ]:
import pickle

In [ ]:
dictionary1File = open(base_dir+"dictionary1.out", "wb")
pickle.dump(dictionary1, dictionary1File)
dictionary1File.close()

dictionary2File = open(base_dir+"dictionary2.out", "wb")
pickle.dump(dictionary2, dictionary2File)
dictionary2File.close()

dictionary3File = open(base_dir+"dictionary3.out", "wb")
pickle.dump(dictionary3, dictionary3File)
dictionary3File.close()

dictionary4File = open(base_dir+"dictionary4.out", "wb")
pickle.dump(dictionary4, dictionary4File)
dictionary4File.close()

dictionary5File = open(base_dir+"dictionary5.out", "wb")
pickle.dump(dictionary5, dictionary5File)
dictionary5File.close()

In [ ]:
dictionary2File = open(base_dir+"dictionary2.out", "wb")
pickle.dump(dictionary2, dictionary2File)
dictionary2File.close()

In [ ]:
dictionary2['trainset_2_1'][0][1]

In [ ]:
### check dictionary2
for i in range(5):
  theTrain = dictionary2["trainset_2_"+str(i+1)]
  total = len(theTrain)
  count = {}
  for k in range(10):
    count[k] = 0
  for item in theTrain:
    key = item[1]
    count[key] += 1
  print('\n',"trainset_2_"+str(i+1), sep='')
  for i in range(10):
    print("percentage of", i, "is: ", count[i]/total*100)

    